# Titanic – Pipeline complète
Chargement → Nettoyage → CSV clean → Modèle → Prédiction Kaggle

## 1. Import des librairies

In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 2. Chargement des CSV bruts

In [21]:
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 3. Nettoyage complet & Feature Engineering

In [22]:
test['Survived'] = np.nan
train['is_train'] = 1
test['is_train'] = 0

full = pd.concat([train, test], ignore_index=True)

full = full.drop(columns=['Name','Ticket','Cabin'])

full['Age'] = full['Age'].fillna(full['Age'].median())
full['Fare'] = full['Fare'].fillna(full['Fare'].median())
full['Embarked'] = full['Embarked'].fillna(full['Embarked'].mode()[0])

full['Sex'] = full['Sex'].map({'male':0,'female':1})
full['FamilySize'] = full['SibSp'] + full['Parch'] + 1
full['IsAlone'] = (full['FamilySize']==1).astype(int)

full = pd.get_dummies(full, columns=['Embarked','Pclass'], drop_first=True)

bool_cols = ['Embarked_Q','Embarked_S','Pclass_2','Pclass_3']
full[bool_cols] = full[bool_cols].astype(int)

train_clean = full[full['is_train']==1].drop(columns=['is_train'])
test_clean = full[full['is_train']==0].drop(columns=['is_train','Survived'])

train_clean.to_csv('../data/clean/train_clean.csv', index=False)
test_clean.to_csv('../data/clean/test_clean.csv', index=False)

train_clean.head()


,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,1,0.0,0,22.0,1,0,7.2500,2,0,0,1,0,1
1,2,1.0,1,38.0,1,0,71.2833,2,0,0,0,0,0
2,3,1.0,1,26.0,0,0,7.9250,1,1,0,1,0,1
3,4,1.0,1,35.0,1,0,53.1000,2,0,0,1,0,0
4,5,0.0,0,35.0,0,0,8.0500,1,1,0,1,0,1


## 4. Split X / y

In [23]:
X = train_clean.drop(columns=['Survived','PassengerId'])
y = train_clean['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape

(712, 11)

## 5. Modèle DecisionTreeClassifier

In [24]:
model = DecisionTreeClassifier(max_depth=4, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
accuracy_score(y_val, y_pred)

0.8044692737430168

## 6. Évaluation du modèle

In [25]:
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

         0.0       0.79      0.94      0.85       110
         1.0       0.85      0.59      0.70        69

    accuracy                           0.80       179
   macro avg       0.82      0.77      0.78       179
weighted avg       0.81      0.80      0.80       179



array([[103,   7],
       [ 28,  41]])

## 7. Prédiction finale Kaggle

DecisionTreeClassifier

In [26]:
X = train_clean.drop(columns=['Survived','PassengerId'])
y = train_clean['Survived']

model = DecisionTreeClassifier(max_depth=4, random_state=42)
model.fit(X, y)

X_test = test_clean.drop(columns=['PassengerId'])
pred = model.predict(X_test).astype(int)


submission = pd.DataFrame({
    "PassengerId": test_clean["PassengerId"],
    "Survived": pred
})

submission.to_csv("../data/result/submission_dtc.csv", index=False)



RandomForestClassifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

X = train_clean.drop(columns=['Survived','PassengerId'])
y = train_clean['Survived']

model = RandomForestClassifier(
    n_estimators=500,
    max_depth=6,
    min_samples_split=4,
    min_samples_leaf=2,
    random_state=42
)

model.fit(X, y)

X_test = test_clean.drop(columns=['PassengerId'])
pred = model.predict(X_test).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_clean['PassengerId'],
    'Survived': pred
})

submission.to_csv('../data/result/submission_rf.csv', index=False)



GradientBoostingClassifier

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
    learning_rate=0.05,
    n_estimators=300,
    max_depth=3,
    random_state=42
)

model.fit(X, y)

pred = model.predict(X_test).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_clean['PassengerId'],
    'Survived': pred
})

submission.to_csv('../data/result/submission_gb.csv', index=False)


XGBoost

In [29]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=400,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X, y)

pred = model.predict(X_test).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_clean['PassengerId'],
    'Survived': pred
})

submission.to_csv('../data/result/submission_xgb.csv', index=False)
